In [5]:
import torch
from torch.fx import symbolic_trace
from typing import Dict


class MyModel(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(10, 10)
        self.linear2 = torch.nn.Linear(10, 10)

    def forward(self, x, y):
        c = self.linear2(x[1] + self.linear((x + y) * 2))
        d = c + 1
        return d


model = MyModel()
gm = symbolic_trace(model)
print(gm)


MyModel(
  (linear): Linear(in_features=10, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=10, bias=True)
)



def forward(self, x, y):
    getitem = x[1]
    add = x + y;  x = y = None
    mul = add * 2;  add = None
    linear = self.linear(mul);  mul = None
    add_1 = getitem + linear;  getitem = linear = None
    linear2 = self.linear2(add_1);  add_1 = None
    add_2 = linear2 + 1;  linear2 = None
    return add_2
    
# To see more debug info, please use `graph_module.print_readable()`


### Split graph into subgraphs

In [6]:
def split_graph(gm):
    num_node = len(gm.graph.nodes)
    num_parts = 3
    node_per_part = num_node // num_parts
    for it, node in enumerate(gm.graph.nodes):
        node.tag = it // node_per_part
    classes = [i for i in range(num_parts)]
    new_gm = torch.fx.passes.split_utils.split_by_tags(gm, classes)
    return new_gm

new_gm = split_graph(gm)
for child in new_gm.children():
    print(child)


GraphModule()



def forward(self, x):
    getitem = x[1];  x = None
    return getitem
    
# To see more debug info, please use `graph_module.print_readable()`
GraphModule(
  (linear): Linear(in_features=10, out_features=10, bias=True)
)



def forward(self, x, y):
    add = x + y;  x = y = None
    mul = add * 2;  add = None
    linear = self.linear(mul);  mul = None
    return linear
    
# To see more debug info, please use `graph_module.print_readable()`
GraphModule(
  (linear2): Linear(in_features=10, out_features=10, bias=True)
)



def forward(self, getitem, linear):
    add_1 = getitem + linear;  getitem = linear = None
    linear2 = self.linear2(add_1);  add_1 = None
    add_2 = linear2 + 1;  linear2 = None
    return add_2
    
# To see more debug info, please use `graph_module.print_readable()`


### Shape propagation

In [7]:
def shape_prop(gm):
    torch.fx.passes.shape_prop.ShapeProp(gm).propagate(torch.randn(10), torch.randn(10))
    return gm

gm = shape_prop(gm)
for node in gm.graph.nodes:
    print(node.name, node.meta['tensor_meta'].dtype,
        node.meta['tensor_meta'].shape)


x torch.float32 torch.Size([10])
y torch.float32 torch.Size([10])
getitem torch.float32 torch.Size([])
add torch.float32 torch.Size([10])
mul torch.float32 torch.Size([10])
linear torch.float32 torch.Size([10])
add_1 torch.float32 torch.Size([10])
linear2 torch.float32 torch.Size([10])
add_2 torch.float32 torch.Size([10])
output torch.float32 torch.Size([10])
